In [1]:
import pandas as pd
import random
import numpy as np
import math
import seaborn as sns
from datetime import date
import datetime
from sklearn import metrics
import six
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 16, 9

In [2]:
data = pd.read_csv('HMXPC13_DI_v2_5-14-14.csv')

In [3]:
data = data.drop(['roles', 'incomplete_flag' , 'registered', 'certified'], axis=1) #We are dropping 3 columns:
#registered becasue it has only value 1 and nothing else; 
#incoplete flag and roles because they have mostly Null values and are only used for administration.

In [4]:
data.shape

(641138, 16)

At the begining of data cleaning we had 641138 rows.

#### We are checking 4 columnes- nevents, ndays_act, nplay_video and nchapters because we want to be sure they have NaN values for the state of no interaction that we can safely replace with the value zero. 

In [5]:
data['nevents'].value_counts(dropna = False).sort_index()

 1.0          63565
 2.0          34329
 3.0          17669
 4.0          12217
 5.0           9850
 6.0           8480
 7.0           7259
 8.0           6648
 9.0           6076
 10.0          5621
 11.0          5197
 12.0          4870
 13.0          4476
 14.0          4222
 15.0          3891
 16.0          3808
 17.0          3545
 18.0          3349
 19.0          3144
 20.0          3064
 21.0          2956
 22.0          2795
 23.0          2578
 24.0          2621
 25.0          2420
 26.0          2391
 27.0          2216
 28.0          2170
 29.0          2122
 30.0          2079
              ...  
 30659.0          1
 30810.0          1
 30849.0          1
 31396.0          1
 31416.0          1
 31761.0          1
 31868.0          1
 31900.0          1
 31904.0          1
 31943.0          1
 32296.0          1
 32498.0          1
 32528.0          1
 32812.0          1
 33650.0          1
 33661.0          1
 33820.0          1
 36244.0          1
 37440.0          1


In [6]:
data['ndays_act'].value_counts(dropna = False).sort_index()

 1.0      209941
 2.0       80625
 3.0       43081
 4.0       26813
 5.0       18552
 6.0       13239
 7.0       10281
 8.0        8075
 9.0        6510
 10.0       5324
 11.0       4415
 12.0       3815
 13.0       3323
 14.0       2794
 15.0       2542
 16.0       2146
 17.0       2002
 18.0       1852
 19.0       1564
 20.0       1497
 21.0       1339
 22.0       1309
 23.0       1207
 24.0       1093
 25.0       1027
 26.0        947
 27.0        887
 28.0        891
 29.0        838
 30.0        806
           ...  
 131.0         4
 132.0         4
 133.0         4
 134.0         3
 135.0         2
 136.0         5
 137.0         1
 138.0         4
 139.0         1
 140.0         1
 141.0         4
 142.0         2
 143.0         4
 144.0         1
 147.0         3
 148.0         1
 149.0         1
 150.0         1
 151.0         1
 153.0         1
 155.0         3
 157.0         1
 158.0         1
 159.0         2
 160.0         1
 162.0         1
 165.0         1
 176.0        

In [7]:
data['nplay_video'].value_counts(dropna = False).sort_index()

 1.0         16968
 2.0         11000
 3.0          8371
 4.0          6995
 5.0          5992
 6.0          5373
 7.0          4714
 8.0          4296
 9.0          4076
 10.0         3620
 11.0         3453
 12.0         3187
 13.0         2853
 14.0         2641
 15.0         2453
 16.0         2401
 17.0         2138
 18.0         2085
 19.0         1887
 20.0         1835
 21.0         1803
 22.0         1671
 23.0         1565
 24.0         1501
 25.0         1553
 26.0         1412
 27.0         1330
 28.0         1219
 29.0         1179
 30.0         1205
             ...  
 8012.0          1
 8049.0          1
 8052.0          1
 8160.0          1
 8223.0          1
 8277.0          1
 8469.0          1
 8632.0          1
 8727.0          1
 9209.0          1
 9310.0          1
 9409.0          1
 9473.0          1
 9478.0          1
 9509.0          1
 9652.0          1
 9754.0          1
 9838.0          1
 10668.0         1
 11458.0         1
 11491.0         1
 12983.0    

In [8]:
data['nchapters'].value_counts(dropna = False).sort_index()

 1.0     121837
 2.0     110085
 3.0      52296
 4.0      24937
 5.0      13838
 6.0       8536
 7.0       6556
 8.0       5009
 9.0       4091
 10.0      3598
 11.0      3258
 12.0      7987
 13.0      2053
 14.0      2021
 15.0      2684
 16.0      2890
 17.0      1877
 18.0      3411
 19.0       473
 20.0       624
 21.0       424
 22.0       150
 23.0       149
 24.0       157
 25.0       250
 26.0       339
 27.0       352
 28.0       193
 29.0       179
 30.0       181
 31.0       385
 32.0       657
 33.0       511
 34.0       271
 35.0        16
 36.0        23
 37.0        15
 38.0         8
 39.0        10
 40.0        12
 41.0         5
 42.0        11
 43.0         6
 44.0         3
 45.0         3
 46.0         3
 47.0         5
 48.0         6
NaN      258753
Name: nchapters, dtype: int64

#### Considering that all 4 tested columns have no value 0 we will now assign 0 to all rows that have NaN value

In [9]:
data['nevents'].fillna(0.0, inplace = True)

In [10]:
data['ndays_act'].fillna(0.0, inplace = True)

In [11]:
data['nplay_video'].fillna(0.0, inplace = True)

In [12]:
data['nchapters'].fillna(0.0, inplace = True)

#### Transforming Country of origin data using get_dummies function - Converts categorical variable into dummy/indicator variables

In [13]:
data = data.join(pd.get_dummies(data['final_cc_cname_DI']))

#### Derive column Total Activity

When the date is missing from the column "last_event_DI" this means that there was no interaction with the course after enrolment. Considering this we have decided to put starting date as the date of last interaction instead of NaN value. This way we keep all entries and we get 0 days active.

In [14]:
data["last_event_DI"] = data.apply(lambda x: x.start_time_DI if pd.isnull(x.last_event_DI) else x.last_event_DI, axis =1)

We have checked the type of all dates entered in columnes "start_time_DI" and "last_event_DI" and they are all String. The code that we used is removed as it takes longer time to run.
#### We now make a derivative columne that shows "total_activity" and it is determined from start time of the course and last event
To work with dates, we need to convert them from strings into a data format built for processing dates. The pandas library comes with a Timestamp data object for storing and working with dates.

In [15]:
data["start_time_DI"] = pd.to_datetime(data["start_time_DI"] )

In [16]:
data["last_event_DI"] = pd.to_datetime(data["last_event_DI"] ) #transfering dates so we can substract them to get days

In [17]:
data['total_activity'] = data['last_event_DI'] - data['start_time_DI']

In [18]:
data['total_activity'] = data['total_activity'].apply(lambda x: x.days)

In [19]:
#data['YoB'].fillna(int(data['YoB'].mean()), inplace = True) #for missing values we set mean of all ages

#### Dealing with NaN values and converting Level of education from categorical variable into indicator variables

In [20]:
data['LoE_DI'] = data['LoE_DI'].fillna('UnknownLoE') #for missing values we assigned Unknown level of Education this is not
# so high precentage of data as shown in the next cell

In [21]:
data['LoE_DI'].value_counts(dropna= False)

Bachelor's             219768
Secondary              169694
Master's               118189
UnknownLoE             106008
Less than Secondary     14092
Doctorate               13387
Name: LoE_DI, dtype: int64

In [22]:
data = data.join(pd.get_dummies(data['LoE_DI']))

#### Dealing with NaN values and converting Gender from categorical variable into indicator variables

In [23]:
data['gender'].fillna('u', inplace = True) #changing NaN values to 'u' -unknown, again this is not too high precentage of data

In [24]:
data['gender'].value_counts(dropna= False)

m    411520
f    142795
u     86806
o        17
Name: gender, dtype: int64

In [25]:
data = data.join(pd.get_dummies(data['gender']))

#### Some grades are empty string (space) and we drop those rows because we can work only with rows that have a grade as indicator of success on course

In [26]:
data = data.loc[data['grade'] != " ",] 

In [27]:
data.dropna(axis=0, inplace=True) #We drop all rows that have NaN after cleaning

#### Transforming Year of birth so we can derive column Age
Based on multiple runs of code and investigations of data we reached a conclusion that columns that have a missing value for year of birth also have missing values for most of other columns like gender and level of education. After trying different options of assigning values and checking results we can say that the best result is when we dont fill in these values but just drop all rows with missing year of birth. From remaining rows we derive column "age".

In [28]:
now = datetime.datetime.now()

In [29]:
data['age'] = data['YoB'].apply(lambda x: now.year - int(x)) # we use provided year of birth to calculate
#age of all participants and save it as new column

In [30]:
data['grade'] = data['grade'].astype(float)

In [31]:
data.columns

Index(['course_id', 'userid_DI', 'viewed', 'explored', 'final_cc_cname_DI',
       'LoE_DI', 'YoB', 'gender', 'grade', 'start_time_DI', 'last_event_DI',
       'nevents', 'ndays_act', 'nplay_video', 'nchapters', 'nforum_posts',
       'Australia', 'Bangladesh', 'Brazil', 'Canada', 'China', 'Colombia',
       'Egypt', 'France', 'Germany', 'Greece', 'India', 'Indonesia', 'Japan',
       'Mexico', 'Morocco', 'Nigeria', 'Other Africa', 'Other East Asia',
       'Other Europe', 'Other Middle East/Central Asia',
       'Other North & Central Amer., Caribbean', 'Other Oceania',
       'Other South America', 'Other South Asia', 'Pakistan', 'Philippines',
       'Poland', 'Portugal', 'Russian Federation', 'Spain', 'Ukraine',
       'United Kingdom', 'United States', 'Unknown/Other', 'total_activity',
       'Bachelor's', 'Doctorate', 'Less than Secondary', 'Master's',
       'Secondary', 'UnknownLoE', 'f', 'm', 'o', 'u', 'age'],
      dtype='object')

In [32]:
data['gender'].value_counts(dropna= False)

m    370685
f    124105
Name: gender, dtype: int64

#### At the end of data cleaning we have 494790 rows which is around 77% of the begining dataset size. Because the number of rows is still significant and most of removed rows had a lot of missing information we consider this a good result.

In [35]:
data.to_csv('cleaned_data.csv', index = False)